In [ ]:
import time
start_time = time.time()

In [ ]:
%%time
import cobra
import pandas as pd
import os
import numpy as np
import yaml
import logging

# Define your custom path here, e.g., "custom/path/to/inputs.yml"
inputs_path = ""

# If inputs_path is empty, use the INPUTS environment variable
if not inputs_path:
    inputs_path = os.getenv("INPUTS")
    if inputs_path is None:
        raise ValueError("The INPUTS environment variable is not set.")

inputs_file = os.path.join(inputs_path, "inputs.yml")
if not os.path.isfile(inputs_file):
    raise FileNotFoundError(f"The 'inputs.yml' file could not be found at {inputs_file}.")

with open(inputs_file, "r") as file:
    data = yaml.safe_load(file)

sbml_model = data["sbml_model"]
test_path = data["output_file_path"]
transporters = data["transporters"]

logging.getLogger('cobra').setLevel(logging.ERROR)
ec_model = cobra.io.read_sbml_model(os.path.join(test_path,'output_GEMs/ec_PAO1_fixed.xml'))
ec_model

In [ ]:
%%time
"""Average reaction coefficient"""

usage_coefficients = []

for reaction in ec_model.reactions:
    if 'resource_usage_pseudometabolite' in [m.name for m in reaction.metabolites]:
        for m in reaction.metabolites:
            if m.id == 'usage':
                coef = reaction.metabolites[m]
                usage_coefficients.append(coef)

average_coef = sum(usage_coefficients)/len(usage_coefficients)
abs(average_coef)

In [ ]:
%%time
patched_model = ec_model.copy()

In [ ]:
%%time
def contains_keywords(cell):
    return any(keyword.lower() in str(cell).lower() for keyword in transporters)

usage = patched_model.metabolites.get_by_id('usage')

excluded_reactions = [
    'Biomass reaction',
    'R_ATPM'
]

with patched_model:
    for reaction in patched_model.reactions:
        if (reaction not in patched_model.boundary and not contains_keywords(reaction.name)
           and reaction.name not in excluded_reactions):
            if ('resource_usage_pseudometabolite' not in [m.name for m in reaction.metabolites]):
                # print(reaction.bounds)
                if reaction.lower_bound < 0 and reaction.upper_bound <= 0:
                    # print('reverse')
                    reaction.add_metabolites({
                        usage: abs(average_coef)
                    })
                if reaction.lower_bound >= 0 and reaction.upper_bound > 0:
                    # print('forward')
                    reaction.add_metabolites({
                        usage: - abs(average_coef)
                    })
                # print(reaction.name, reaction.reaction)

    cobra.io.write_sbml_model(patched_model, os.path.join(test_path,'output_GEMs/patched_ec_PAO1_fixed.xml'))

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Total execution time: {elapsed_time:.2f} seconds')